# Qrack Quantum Fourier Transform Validation

How do we know that a GHZ state is Qrack's hardest initialization case for the QFT? We can test this, with random circuits. However, it helps to explain our best guesses at hypothetical hardest cases, first.

Because Qrack attempts to factorize its interacting qubit systems to the smallest possible separable subsystems, we anticipate two major factors to contribute to overhead scaling: overhead scales exponentially with subsystem width, but combining factorized subsystems (to interact) also requires a **Kronecker product** operation, which is a source of overhead "naive" state vector simulation never needs at all.

Hence, there are two obvious (nonexclusive) possibilities to consider for the most expensive case. The most expensive case might be any that starts as a full-width, unfactorized state vector simulation, since it derives no benefit relative to "naive" state vector simulation, from Qrack's substate factorization capabilities. Alternatively, it's plausible that a state that starts _fully_ factorized into single qubits, but which we can be sure to _end up as a full-width state vector_, might pay more cost in Kronecker product overhead than overhead saved by any temporary factorization.

The second case might actually be worse than GHZ at middle and low qubit widths, relative the maximum allocation segment of a typical 2023 NVIDIA GeForce GPU. GHZ has been found to be the worse of the two at higher widths.

However, can we randomly guess any harder cases than these, with RCS? (We try this, below.)

In [1]:
low = 1
high = 27
samples = 10

In [2]:
%env QRACK_QUNITMULTI_DEVICES 1
import random
import time
from pyqrack import QrackSimulator, Pauli

env: QRACK_QUNITMULTI_DEVICES=1


In [3]:
def reverse_qrack(sim):
    start = 0
    end = sim.num_qubits() - 1
    while (start < end):
        sim.swap(start, end)
        start += 1
        end -= 1

def qrack_qft(sim, n):
    sim.iqft([i for i in reversed(range(n))])
    reverse_qrack(sim)
    sim.m_all()

def rand_u3(sim, q):
    th = random.uniform(0, 4 * math.pi)
    ph = random.uniform(0, 4 * math.pi)
    lm = random.uniform(0, 4 * math.pi)
    sim.u(q, th, ph, lm)

## GHZ initialization

In [4]:
def bench_ghz_qrack(n):
    sim = QrackSimulator(n)
    sim.set_reactive_separate(False)

    # GHZ init
    sim.h(0)
    for i in range(n - 1):
        sim.mcx([i], i + 1)

    # Block and complete queued operations by reading output on all qubits.
    for i in range(n):
        sim.prob(i)

    # Start timer and run (inverse) QFT
    start = time.perf_counter()
    qrack_qft(sim, n)

    return time.perf_counter() - start

qrack_ghz_results = {}
for n in range(low, high + 1):
    width_results = []
        
    # Run the benchmarks
    for i in range(samples):
        width_results.append(bench_ghz_qrack(n))

    qrack_ghz_results[n] = sum(width_results) / samples

print(qrack_ghz_results)

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_Intel(R)_UHD_Graphics_[0x9bc4].ir
Device #1, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir
{1: 1.1164399984409101e-05, 2: 3.2996599838952537e-05, 3: 0.00013319720037543447, 4: 0.00024407750024693086, 5: 0.00015133670003706357, 6: 0.00014174159969115863, 7: 0.0001772409998011426, 8: 0.000225597299686342, 9: 0.00030800079985056074, 10: 0.0004731696997623658, 11: 0.0015640158999303821, 12: 0.0018655242998647736, 13: 0.0021405497998784996, 14: 0.0025488383003903436, 15: 0.002910359899942705, 16: 0.0033808856998803092, 17: 0.004278139100097178, 18: 0.003591272799712897, 19: 0.0045192783001766655, 20: 0.008387881699673017, 21: 0.0157108478997543, 22: 0.029210087600040423, 23: 0.054105507300300816, 24: 0.10919690820028335, 25: 0.22117784779984503, 26: 0.45902814910023154, 27: 0.9660312883001098}


## Randomized single separable qubits initialization

In [5]:
def bench_u3_qrack(n):
    sim = QrackSimulator(n)
    sim.set_reactive_separate(False)

    # U3 init
    for i in range(n):
        rand_u3(sim, i)

    # Block and complete queued operations by reading output on all qubits.
    for i in range(n):
        sim.prob(i)

    # Start timer and run (inverse) QFT
    start = time.perf_counter()
    qrack_qft(sim, n)

    return time.perf_counter() - start

qrack_u3_results = {}
for n in range(low, high + 1):
    width_results = []
        
    # Run the benchmarks
    for i in range(samples):
        width_results.append(bench_u3_qrack(n))

    qrack_u3_results[n] = sum(width_results) / samples

print(qrack_u3_results)

NameError: name 'math' is not defined

## Random circuit initialization

In [ ]:
import random
import math
import numpy as np

In [ ]:
#RCS gates

def cx(sim, q1, q2):
    sim.mcx([q1], q2)

def cy(sim, q1, q2):
    sim.mcy([q1], q2)

def cz(sim, q1, q2):
    sim.mcz([q1], q2)

def acx(sim, q1, q2):
    sim.macx([q1], q2)

def acy(sim, q1, q2):
    sim.macy([q1], q2)

def acz(sim, q1, q2):
    sim.macz([q1], q2)

def swap(sim, q1, q2):
    sim.swap(q1, q2)

two_bit_gates = cx, cz, cy, acx, acz, acy

In [ ]:
def bench_rcs_qrack(n):
    sim = QrackSimulator(n)
    sim.set_reactive_separate(False)

    # RCS init
    for i in range(n):
        # Single bit gates
        for j in range(n):
            rand_u3(sim, j)

        # Multi bit gates
        bit_set = [i for i in range(n)]
        while len(bit_set) > 1:
            b1 = random.choice(bit_set)
            bit_set.remove(b1)
            b2 = random.choice(bit_set)
            bit_set.remove(b2)
            gate = random.choice(two_bit_gates)
            gate(sim, b1, b2)

    # Block and complete queued operations by reading output on all qubits.
    for i in range(n):
        sim.prob(i)

    # Start timer and run (inverse) QFT
    start = time.perf_counter()
    qrack_qft(sim, n)

    return time.perf_counter() - start

qrack_rcs_results = {}
for n in range(low, high + 1):
    width_results = []
        
    # Run the benchmarks
    for i in range(samples):
        width_results.append(bench_rcs_qrack(n))

    qrack_rcs_results[n] = sum(width_results) / samples

print(qrack_rcs_results)

## Results

In [ ]:
import matplotlib.pyplot as plt

fig = plt.gcf()
fig.set_size_inches(14, 14)
plt.rc('legend',fontsize=28)
plt.rcParams.update({'font.size': 28, 'lines.markersize': 12})

colors = list("kcymbgr")
markers = list("D*PX^so")

x = qrack_ghz_results.keys()
y = qrack_ghz_results.values()
plt.scatter(x,y,color=colors.pop(),marker=markers.pop())

x = qrack_u3_results.keys()
y = qrack_u3_results.values()
plt.scatter(x,y,color=colors.pop(),marker=markers.pop())

x = qrack_rcs_results.keys()
y = qrack_rcs_results.values()
plt.scatter(x,y,color=colors.pop(),marker=markers.pop())

plt.title("N-qubit (inverse) QFT, PyQrack")
plt.xlabel("Circuit width (qb)")
plt.ylabel("Time (s)")
plt.legend(["GHZ", "U3", "RCS"])
plt.yscale("log")
plt.xticks(np.arange(low, high + 1, step=2))

plt.show()

...In the end, these cases are probably all close to about the hardest we can find. (Different variants of GHZ might also be harder than our supplied and tested case, as wall.)